In [ ]:
using ITensors, ITensorMPS, ITensorGaussianMPS, LinearAlgebra, Plots

# Function to construct the hopping + interaction Hamiltonian matrix
function hopping_matrix(N::Int, t1::Float64, t2::Float64, V::Float64)
    H = zeros(Float64, N, N)
    
    # Nearest-neighbor hopping terms (-t)
    for j in 1:N-1
        H[j, j+1] = -t1
        H[j+1, j] = -t2
    end
    # Periodic boundary condition for hopping
    H[1, N] = -t2
    H[N, 1] = -t2

    # Interaction term (-V n_j n_{j+1}) (Mean-field approximation)
    for j in 1:N-1
        H[j, j] += -V  # Diagonal interaction terms
        H[j+1, j+1] += -V
    end
    # Periodic boundary condition for interaction
    H[1, 1] += -V
    H[N, N] += -V

    return H
end
# Function to compute the Slater determinant matrix
function compute_slater_matrix(H::Matrix{Float64}, Nf::Int)
    eigvals, eigvecs = eigen(H)
    return eigvecs[:, 1:Nf]  # Take lowest-energy Nf eigenvectors
end

# Function to run DMRG on the interacting SSH model
function run_dmrg_ssh(N::Int, t1::Float64, t2::Float64, V::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Float64, noise::Float64, weight::Float64)
    _maxlinkdim = 1300
    _cutoff = 1e-10
    _eigval_cutoff = 1e-8
    _maxblocksize = 30 
    Nf = N ÷ 2  # Half-filled fermion system

    # Construct free fermion hopping Hamiltonian
    H_free = hopping_matrix(N, t)
    
    # Compute Slater determinant matrix
    Φ = compute_slater_matrix(H_free, Nf)
    
    # Define fermionic sites
    sites = siteinds("Fermion", N; conserve_qns = true)
    
    println("Making free fermion starting MPS")
    @time psi0_init = slater_determinant_to_mps(sites, Φ; eigval_cutoff=_eigval_cutoff, cutoff=_cutoff, maxdim=_maxlinkdim, maxblocksize=_maxblocksize)

    # Define SSH model Hamiltonian (free part)
    os = OpSum()
    for j in 1:N-1
        hopping = j % 2 == 1 ? t1 : t2  # Alternating hopping strengths
        os += -hopping, "Cdag", j, "C", j + 1
        os += -hopping, "Cdag", j + 1, "C", j
    end
    
    # Periodic boundary condition for hopping (t2)
    os += -t2, "Cdag", 1, "C", N
    os += -t2, "Cdag", N, "C", 1

    # Define interacting term (-V n_j n_{j+1})
    os_interacting = OpSum()  
    for j in 1:N-1
        os_interacting += -V, "N", j, "N", j+1
    end
    os_interacting += -V, "N", N, "N", 1  # Periodic BC

    # Convert OpSum to MPO
    H = MPO(os + os_interacting, sites)

    # Perform DMRG calculations
    println("Running DMRG for ground state")
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    println("Running DMRG for first excited state")
    energy_excited1, psi_excited1 = dmrg(H, [psi_ground], psi0_init; nsweeps, maxdim, cutoff, noise, weight)

    println("Running DMRG for second excited state")
    energy_excited2, psi_excited2 = dmrg(H, [psi_ground, psi_excited1], psi0_init; nsweeps, maxdim, cutoff, noise, weight)

    # Compute Energy Gaps
    gap_1 = energy_excited1 - energy_ground
    gap_2 = energy_excited2 - energy_ground

    println("V = $V, Ground Energy: $energy_ground, First Excited Energy: $energy_excited1, Second Excited Energy: $energy_excited2")
    println("First Energy Gap: $gap_1, Second Energy Gap: $gap_2")
    
    return (gap_1, gap_2)
end

# Parameters
N = 24  # Number of sites   
t1 = 1.0  
t2 = 0.8
V_values = collect(-4.6:0.4:6.0)  # Smooth range
nsweeps = 10
maxdim = [10, 20, 50, 100, 200, 400]
noise = 1E-10
cutoff = 1E-10 
weight = 5.0 

# Compute energy gaps

energy_gaps = [run_dmrg_ssh(N, t1, t2, V, nsweeps, maxdim, cutoff, noise, weight) for V in V_values]

# Extract first and second energy gaps
gap_1_values = [gaps[1] for gaps in energy_gaps]
gap_2_values = [gaps[2] for gaps in energy_gaps]

# Plot results
plot(V_values, gap_1_values, marker=:o, linewidth=2, label="1st Energy Gap", color=:blue)
plot!(V_values, gap_2_values, marker=:s, linewidth=2, label="2nd Energy Gap", color=:red)
vline!([-2.5], linestyle=:dash, color=:gray, label="V = -2.5")

xlabel!("V")
ylabel!("Energy Gap")
title!("Energy Gaps vs Interaction V")

# Save and display the plot
savefig("ssh_interacting_engaps_plot_test_osama.pdf")
display(plot!)